In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import pickle
import os
import faiss
from openai import OpenAI
import numpy as np

## 1. Crawl data from llama-index blogs and preprocessing

In [ ]:
import requests
from bs4 import BeautifulSoup
import pickle
import os
import faiss
from openai import OpenAI
import numpy as np
from tqdm import tqdm

In [ ]:
# Set up headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Making a GET request with headers
try:
    r = requests.get('https://www.llamaindex.ai/blog', headers=headers, timeout=10)
    r.raise_for_status()  # Raise an exception for bad status codes

    # Print status code
    print(f"Status Code: {r.status_code}")

    # Parsing the HTML
    soup = BeautifulSoup(r.content, 'html.parser')

    # Print the title of the page
    print(f"Page Title: {soup.title.string if soup.title else 'No title found'}")

except requests.RequestException as e:
    print(f"An error occurred: {e}")

Status Code: 200
Page Title: Blog — LlamaIndex, Data Framework for LLM Applications


In [ ]:
def extract_blog_content(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    r = requests.get(url, headers=headers, timeout=10)

    # Parsing the HTML content using BeautifulSoup
    soup = BeautifulSoup(r.content, 'html.parser')

    # Extract the blog title
    blog_title = soup.find('h1')
    if blog_title:
        blog_title = blog_title.text.strip()
    else:
        blog_title = "No title found"

    # Extract the blog content
    content_div = soup.find('div', class_='BlogPost_htmlPost__Z5oDL')

    chunks = []

    count_session = 0

    if content_div.find(['h2']) == None:
        session_type = 'h3'
        content_type = ['h3','p','blockquote','li','pre','figcaption']
    else:
        session_type = 'h2'
        content_type = ['h2','h3','p','blockquote','li','pre','figcaption']

    if content_div:
        current_chunk = None
        paragraphs = content_div.find_all(content_type)

        for paragraph in paragraphs:
            if paragraph.name == session_type:
                if current_chunk:
                    final_chunk = f"""
                    Title: {current_chunk['title']}
                    -----------
                    Session title: {current_chunk['session_title']}
                    -----------
                    Content: {current_chunk['content']}
                    """
                    chunks.append(final_chunk)

                current_chunk = {
                    'title': blog_title,
                    'session_title': paragraph.text.strip(),
                    'content': ''
                }
                count_session+=1
            elif count_session > 0:
                # Clean and add text of each paragraph to the current chunk content
                text = paragraph.text.strip()
                if text:  # Make sure not to add empty strings
                    current_chunk['content'] += '\n' + text
            else:
                text = paragraph.text.strip()
                if current_chunk:
                    if text:
                        current_chunk['content'] += '\n' + text
                else:
                    current_chunk = {
                        'title': blog_title,
                        'session_title': None,
                        'content': text
                    }

        if current_chunk:
            final_chunk = f"""
            Title: {current_chunk['title']}
            -----------
            Session title: {current_chunk['session_title']}
            -----------
            Content: {current_chunk['content']}
            """
            chunks.append(final_chunk)

    return chunks
# # Example usage
url = 'https://www.llamaindex.ai/blog/one-click-open-source-rag-observability-with-langfuse'
blog_chunks = extract_blog_content(url)
for chunk in blog_chunks:
    print(chunk)
    break


                    Title: One-click Open Source RAG Observability with Langfuse
                    -----------
                    Session title: None
                    -----------
                    Content: This is a guest post from the team at Langfuse
There are so many different ways to make RAG work for a use case. What vector store to use? What retrieval strategy to use? LlamaIndex makes it easy to try many of them without having to deal with the complexity of integrations, prompts and memory all at once.
Initially, we at Langfuse worked on complex RAG/agent applications and quickly realized that there is a new need for observability and experimentation to tweak and iterate on the details. In the end, these details matter to get from something cool to an actually reliable RAG application that is safe for users and customers. Think of this: if there is a user session of interest in your production RAG application, how can you quickly see whether the retrieved context for tha

In [ ]:
# Assuming the HTML content is stored in a variable called 'html_content'
# soup = BeautifulSoup(soup, 'html.parser')

# Find all blog post cards
r = requests.get('https://www.llamaindex.ai/blog')
soup = BeautifulSoup(r.content, 'html.parser')

blog_cards = soup.find_all('div', class_='CardBlog_card__mm0Zw')
base_url = "https://www.llamaindex.ai"

# List of crawl content
crawl_content = []

# Extract and print the main text from each card
for card in tqdm(blog_cards):
    # Extract title
    title_element = card.find('p', class_='CardBlog_title__qC51U').find('a')
    url = base_url + title_element['href']

    blog_chunks = extract_blog_content(url)

    for chunk in blog_chunks:
        crawl_content.append(chunk)

if not(os.path.exists('crawl_content.pkl')):
  # store the crawl content dictionary to a file
  with open('crawl_content.pkl', 'wb') as crawl_file:
      pickle.dump(crawl_content, crawl_file)
else:
  with open('crawl_content.pkl', 'rb') as crawl_file:
      crawl_content = pickle.load(crawl_file)

100%|██████████| 157/157 [00:59<00:00,  2.66it/s]


In [ ]:
with open('../API_key', 'r') as f:
    os.environ['OPENAI_API_KEY'] = f.read()

# Initialize an OpenAI instance
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def get_text_embeddings(input):
    embeddings_batch_response = client.embeddings.create(
        model='text-embedding-3-small',
        input = input
    )
    return embeddings_batch_response.data[0].embedding

if not(os.path.exists('embeddings.pkl')):
  text_embeddings = []
  for content in tqdm(crawl_content):
      content_embedding = get_text_embeddings(content)
      text_embeddings.append(content_embedding)
  text_embeddings = np.array(text_embeddings)

  # Save embeddings to a file
  with open('embeddings.pkl', 'wb') as f:
      pickle.dump(text_embeddings, f)
else:
  with open('embeddings.pkl', 'rb') as f:
      text_embeddings = pickle.load(f)

print('Shape of text embedding: ',text_embeddings.shape)


In [ ]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [ ]:
question = 'What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system?'
question_embeddings = np.array([get_text_embeddings(question)])
print('Question embedding shape: ', question_embeddings.shape)

Question embedding shape:  (1, 1536)


In [ ]:
D, I = index.search(question_embeddings, k=1)
print(I)
print(D)

[[419 420]]
[[0.76932764 0.80099964]]


In [ ]:
retrieved_chunk = [crawl_content[i] for i in I.tolist()[0]]
print(len(retrieved_chunk), retrieved_chunk)

In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

print(prompt)


Context information is below.
---------------------
['\n                    Title: Boosting RAG: Picking the Best Embedding & Reranker models\n                    -----------\n                    Session title: Impact of Rerankers:\n                    -----------\n                    Content: \nWithoutReranker: This provides the baseline performance for each embedding.\nbge-reranker-base: Generally improves both hit rate and MRR across embeddings.\nbge-reranker-large: This reranker frequently offers the highest or near-highest MRR for embeddings. For several embeddings, its performance rivals or surpasses that of the CohereRerank.\nCohereRerank: Consistently enhances performance across all embeddings, often providing the best or near-best results.    \n                    ']
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system?
A

In [ ]:
def run_llm(user_message, model="gpt-3.5-turbo"):
    messages = [
        dict(role="user", content=user_message)
    ]
    chat_response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

from time import time

ts = time()
answer = run_llm(prompt)
print(answer)

print('Time: ', time() - ts)

The two main metrics used to evaluate the performance of the different rerankers in the RAG system are hit rate and MRR (Mean Reciprocal Rank).
Time:  1.470202922821045


### 2. Building RAG System

In [ ]:
# Load the crawl dict from the file
with open('crawl_content.pkl', 'rb') as crawl_file:
    crawl_content = pickle.load(crawl_file)

In [ ]:
with open('../API_key', 'r') as f:
    os.environ['OPENAI_API_KEY'] = f.read()

# Initialize an OpenAI instance
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def get_text_embeddings(input):
    embeddings_batch_response = client.embeddings.create(
        model='text-embedding-3-small',
        input = input
    )
    return embeddings_batch_response.data[0].embedding

In [ ]:
# Load embeddings from file
with open('embeddings.pkl','rb') as f:
    text_embeddings = pickle.load(f)

print('Shape of text embedding: ',text_embeddings.shape)

Shape of text embedding:  (649, 1536)


In [ ]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [ ]:
question = 'What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system? '
question_embeddings = np.array([get_text_embeddings(question)])
print('Question embedding shape: ', question_embeddings.shape)

D, I = index.search(question_embeddings, k=1)
print(I)
print(D)

retrieved_chunk = [crawl_content[i] for i in I.tolist()[0]]
print(len(retrieved_chunk), retrieved_chunk)

Question embedding shape:  (1, 1536)
[[419]]
[[0.76145077]]
1 ['\n                    Title: Boosting RAG: Picking the Best Embedding & Reranker models\n                    -----------\n                    Session title: Impact of Rerankers:\n                    -----------\n                    Content: \nWithoutReranker: This provides the baseline performance for each embedding.\nbge-reranker-base: Generally improves both hit rate and MRR across embeddings.\nbge-reranker-large: This reranker frequently offers the highest or near-highest MRR for embeddings. For several embeddings, its performance rivals or surpasses that of the CohereRerank.\nCohereRerank: Consistently enhances performance across all embeddings, often providing the best or near-best results.    \n                    ']


In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

print(prompt)


Context information is below.
---------------------
['\n                    Title: Boosting RAG: Picking the Best Embedding & Reranker models\n                    -----------\n                    Session title: Impact of Rerankers:\n                    -----------\n                    Content: \nWithoutReranker: This provides the baseline performance for each embedding.\nbge-reranker-base: Generally improves both hit rate and MRR across embeddings.\nbge-reranker-large: This reranker frequently offers the highest or near-highest MRR for embeddings. For several embeddings, its performance rivals or surpasses that of the CohereRerank.\nCohereRerank: Consistently enhances performance across all embeddings, often providing the best or near-best results.    \n                    ']
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system? 


In [ ]:
def run_llm(user_message, model="gpt-3.5-turbo"):
    messages = [
        dict(role="user", content=user_message)
    ]
    chat_response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

from time import time

ts = time()
answer = run_llm(prompt)
print(answer)

print('Time: ', time() - ts)

The two main metrics used to evaluate the performance of the different rerankers in the RAG system are hit rate and Mean Reciprocal Rank (MRR).
Time:  1.1446759700775146


## 3. Evaluation

In [ ]:
# Question/Context/Groundtruth
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Document

# generator with openai models
generator_llm = OpenAI(model="gpt-3.5-turbo-16k")
critic_llm = OpenAI(model="gpt-4")
embeddings = OpenAIEmbedding()

generator = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings,
)

C:\Users\khanhthu\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_files=["attention.pdf"]).load_data()
print(documents[0].metadata)

{'page_label': '1', 'file_name': 'attention.pdf', 'file_path': 'attention.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-07-21', 'last_modified_date': '2024-07-12'}


In [ ]:
import pickle
with open('crawl_content.pkl', 'rb') as f:
    crawl_content = pickle.load(f)
documents = [Document(text=content) for content in crawl_content]
print(documents[0])

Doc ID: 2be60230-2622-4b6f-9353-452e5e55e456
Text: Title: Case Study: How Scaleport.ai Accelerated Development and
Improved Sales with LlamaCloud                     -----------
Session title: The Challenge: Streamlining AI Development
-----------                     Content:  Scaleport AI specializes in
transforming emerging AI technology into tangible bu...
